#  Few-shot распознавание сущностей

Для обучения ner-моделей обычно нужно много данных. Но есть способ сократить требование к объёму данных - использовать предобученные языковые модели вроде Берта. Из-за того, что языковая модель даёт хорошие эмбединги, можно завести ner всего с 10 предложений.

Такой подхол называется Few-shot learning. Подробнее с тем, как это работает, мы разберемся, когда будем проходить нейроныые сети. А пока научимся пользоваться готовыми штуками, чтобы решать практические задачи. 

Берт и возможность тренировать few-shot ner модели есть в deep pavlov (http://docs.deeppavlov.ai/en/master/features/models/ner.html)

In [1]:
!pip install deeppavlov

     |████████████████████████████████| 993kB 6.6MB/s 
     |████████████████████████████████| 7.3MB 11.1MB/s 
     |████████████████████████████████| 655kB 34.8MB/s 
     |████████████████████████████████| 6.7MB 34.8MB/s 
     |████████████████████████████████| 20.1MB 1.4MB/s 
     |████████████████████████████████| 1.5MB 40.1MB/s 
     |████████████████████████████████| 8.2MB 34.2MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 10.4MB 39.2MB/s 
     |████████████████████████████████| 2.1MB 41.2MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 512kB 38.1MB/s 
     |████████████████████████████████| 860kB 43.6MB/s 
     |

In [2]:
!pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 34kB/s 
     |████████████████████████████████| 512kB 42.4MB/s 
     |████████████████████████████████| 3.8MB 40.1MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=bb6a7abb4255de035049df294f5b2023e058a46dd43e1b4c6fd21ca7d5e44d0b
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

In [3]:
!apt-get --yes install git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.17.1-1ubuntu0.7).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [4]:
!pip install git+https://github.com/deepmipt/bert.git@feat/multi_gpu

  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-req-build-2qakuvkn
  Running command git clone -q https://github.com/deepmipt/bert.git /tmp/pip-req-build-2qakuvkn
  Created wheel for bert-dp: filename=bert_dp-1.0-cp37-none-any.whl size=23581 sha256=ea18e9976192ae8183d76fa6dac4595744270c5e0c95a7c711d7384d66a4a24a
  Stored in directory: /tmp/pip-ephem-wheel-cache-1dh8inff/wheels/1e/41/94/886107eaf932532594886fd8bfc9cb9d4db632e94add49d326
Successfully built bert-dp


In [5]:
# загрузка файлов для колаба
# from google.colab import files
# uploaded = files.upload()

In [6]:
# в колабе файлы не перезаписываются, создается новый с (1,2,3..). 
# Чтобы удалить старый файл перед загрузкой нового используйте rm file
# rm .txt удаляет все txt файлы в текущей папке
#!rm *.txt

In [1]:
import pandas as pd

Возьмем тексты с авито по категории приставки. 

In [2]:
data = pd.read_csv('pristavki.csv', header=None, names=['text'])

Чтобы научиться выделять названия разметим небольшое количество текстов. Для DeepPavlov разметку нужно поделить на три файла: train.txt, valid.txt, test.txt.

In [3]:
!ls *.txt

test.txt  train.txt  valid.txt


Файлы для обучения, валидации и тестирования в формате IOB. B маркирует начало сущности, а I продолжение. O - обозначает несущность. PRS - это сокращение от PRISTAVKA. Можно было и полностью написать. 

In [4]:
!head train.txt

Продам O
PS3 B-PRS
, O
2 O
беспроводных O
джойстика O
в O
комплекте O

Продам O


Попробуем теперь обучить разметчик сущностей поверх многоязычного берта. Знать, что это такое и как работает не обязательно. Обязательно указать путь к обучающий файлам.

In [5]:
import json
from deeppavlov import configs, build_model, train_model

with configs.ner.ner_ontonotes_bert_mult.open(encoding='utf8') as f:
    ner_config = json.load(f)

ner_config['dataset_reader']['data_path'] = './'  # directory with train.txt, valid.txt and test.txt files
ner_config['metadata']['variables']['NER_PATH'] = './'
ner_config['metadata']['download'] = [ner_config['metadata']['download'][-1]]  # do not download the pretrained ontonotes model



!!! Если колаб крашиться из-за памяти, то после перезапуска не запускайте ячейку с train_model и build_model по очереди. Либо запустите только train_model, либо пропустите ее и переходите к build_model ниже

In [ ]:
ner_model = train_model(ner_config, download=True)

Обучается оно около 10 минут. Добавление примеров в обучение увеличит время обучения. 

Посмотрим как это все размечается.

In [11]:
ner_model(['Playstation 4', 'Xbox 360 продам', 'Продам ПС 3'])

[[['Playstation', '4'], ['Xbox', '360', 'продам'], ['Продам', 'ПС', '3']],
 [['B-PRS', 'I-PRS'], ['B-PRS', 'I-PRS', 'O'], ['O', 'O', 'O']]]

In [12]:
marked = []

for text in data.text.values[:1000]:
    # BERT имеет лимит на длину текста в 512 слов, возьмем даже еще меньше
    if len(text.split()) > 100:
        continue
    pred = ner_model([text])
    sent, tags = pred[0][0], pred[1][0]
    
    # достанем только тексты с сущностями
    if len(set(tags[0])) > 1:
        marked.append(list(zip(sent,tags)))

In [13]:
marked

[[('Sony', 'B-PRS'),
  ('PS3', 'I-PRS'),
  ('в', 'O'),
  ('отличном', 'O'),
  ('состоянии', 'O'),
  (',', 'O'),
  ('продаю', 'O'),
  ('за', 'O'),
  ('ненадобностью', 'O'),
  (',', 'O'),
  ('память', 'O'),
  ('12gb', 'O'),
  ('-', 'O'),
  ('встроенные', 'O'),
  ('+', 'O'),
  ('жесткий', 'O'),
  ('диск', 'O'),
  ('на', 'O'),
  ('160gb', 'O'),
  (',', 'O'),
  ('в', 'O'),
  ('комплект', 'O'),
  ('входит', 'O'),
  ('два', 'O'),
  ('джойстика', 'O'),
  ('(', 'O'),
  ('состояние', 'O'),
  ('новых', 'O'),
  (',', 'O'),
  ('не', 'O'),
  ('залипают', 'O'),
  (',', 'O'),
  ('не', 'O'),
  ('тупят', 'O'),
  (',', 'O'),
  ('играли', 'O'),
  ('мало', 'O'),
  (')', 'O'),
  (',', 'O'),
  ('игра', 'O'),
  ('FIFA', 'O'),
  ('2014', 'O'),
  (',', 'O'),
  ('кабель', 'O'),
  ('hdmi', 'O'),
  (',', 'O'),
  ('коробка', 'O'),
  ('.', 'O')],
 [('Battlefield', 'B-PRS'),
  ('4', 'I-PRS'),
  (',', 'O'),
  ('Granturismo', 'O'),
  ('3', 'I-PRS'),
  ('и', 'O'),
  ('Звезды', 'O'),
  ('Playstation', 'O'),
  ('Битва', '

Приставки конечно размечаются, но ещё и другие слова на английском начали захватываться. Можно попробовать добавить в данные примеры с ними, или сделать даже ещё пару типов сущностей, чтобы модель научилась их разделять.

В этой же моделе есть несколько стандартных сущностей.

!!! Если колаб крашиться из-за памяти, то после перезапуска не запускайте ячейку с train_model и build_model по очереди. Либо запустите только train_model, либо пропустите ее и переходите к build_model

In [6]:
ner_model = build_model(configs.ner.ner_ontonotes_bert_mult, download=True)

2021-03-11 09:42:02.924 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip to /root/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip
100%|██████████| 663M/663M [03:21<00:00, 3.28MB/s]
2021-03-11 09:45:25.808 INFO in 'deeppavlov.core.data.utils'['utils'] at line 268: Extracting /root/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip archive into /root/.deeppavlov/downloads/bert_models
2021-03-11 09:45:34.96 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/deeppavlov_data/ner_ontonotes_bert_mult_v1.tar.gz to /root/.deeppavlov/ner_ontonotes_bert_mult_v1.tar.gz
100%|██████████| 1.32G/1.32G [05:11<00:00, 4.24MB/s]
2021-03-11 09:50:46.121 INFO in 'deeppavlov.core.data.utils'['utils'] at line 268: Extracting /root/.deeppavlov/ner_ontonotes_bert_mult_v1.tar.gz archive into /root/.deeppavlov/models
[nltk_data] Downloading package p

2021-03-11 09:51:07.723 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_ontonotes_bert_mult/tag.dict]











The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API




Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use sta

2021-03-11 09:51:41.867 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/ner_ontonotes_bert_mult/model]



INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_ontonotes_bert_mult/model


In [7]:
ner_model(['Передай привет Михаилу Нефедову'])

[[['Передай', 'привет', 'Михаилу', 'Нефедову']],
 [['O', 'O', 'B-PERSON', 'I-PERSON']]]

In [8]:
ner_model(['Поезжай в Москву'])

[[['Поезжай', 'в', 'Москву']], [['O', 'O', 'B-GPE']]]

In [9]:
ner_model(['Поезжай в Альпы'])

[[['Поезжай', 'в', 'Альпы']], [['O', 'O', 'B-LOC']]]

In [10]:
ner_model(['Я говорю на русском и английском языках'])

[[['Я', 'говорю', 'на', 'русском', 'и', 'английском', 'языках']],
 [['O', 'O', 'O', 'B-LANGUAGE', 'O', 'B-LANGUAGE', 'O']]]

In [11]:
ner_model(['Бутылка воды стоит 50 рублей'])

[[['Бутылка', 'воды', 'стоит', '50', 'рублей']],
 [['O', 'O', 'O', 'B-MONEY', 'I-MONEY']]]

Но он конечно не идеальный.

In [12]:
ner_model(['ле Биг мак'])

[[['ле', 'Биг', 'мак']], [['B-PERSON', 'I-PERSON', 'I-PERSON']]]